athena2pyspark
==

“La documentación es como el sexo; cuando es bueno, es muy, muy bueno, y cuando es malo, es mejor que nada” 
-- Dick Brandon

Usted está frente athena2pyspark una api creada por exalítica para la manipulación de consultas en sql desde s3 basado en athena. El objetivo de esta librería es no tener un futuro sin servidores y sin cluster EMR. Cuide esta librería, apoye reportando bugs y disfrútela.

Este código está siendo probado constantemente en distintos ambientes, de forma local y en jobs de glue ...

Instalación
==
Si usted quiere probar la librería localmente no tiene más que instalarla

pip install git+https://ljofre-exalitica@bitbucket.org/exalitica-team/athena2pyspark.git

Si quiere usarla en un job de glue, debe apuntar a s3://library.exalitica.com/athena2pyspark.zip 

In [1]:
import athena2pyspark as ath 
from athena2pyspark.config import getLocalSparkSession

spark = getLocalSparkSession() # se demora un poco porque esta creado el SparkSession ...

In [2]:
spark

por ejemplo, tenemos la función que retorna la consulta de producto nuevo

Hacer una query y que te retorne un dataframe
==

Podemos usar los recursos de athena para obtener querys basado en sql, esto nos permite hacer querys gigantes de forma server-less

In [3]:
s3_output = "s3://leonardo.exalitica.com/glue_example/query_examples_select_all/"
path_dataframe = ath.run_query(query = "select * from baul_2 limit 10", 
                               database = "ljofre", 
                               s3_output = s3_output,
                               spark = spark)

Execution ID: b5237524-0fe0-48c7-9d6c-1efd435da76b


In [11]:
df = ath.get_dataframe(path_query=path_dataframe, spark=spark)
df.show()

+---------+-----+-----+------+-------+---------+----------+----------+-----+------+-------+----+----+-----------------+-------+
| party_id| corr|cv_us|cv_u4s|cv_u12s|     m_us|     m_u4s|    m_u12s|cp_us|cp_u4s|cp_u12s| rec|tu4s|            tu12s|  n_key|
+---------+-----+-----+------+-------+---------+----------+----------+-----+------+-------+----+----+-----------------+-------+
|159752880| 7772|    2|     2|      7|1847.0588| 1847.0588| 8265.5461|  2.0|   2.0|    9.0| 8.0| 4.0|8.166666666666666|2016_10|
|100151227|27810|    0|     0|      1|      0.0|       0.0| 2586.5545|  0.0|   0.0|   0.44|46.0|30.0|             90.0|2016_10|
|159948119| 2728|    0|     0|      2|      0.0|       0.0| 3756.4225|  0.0|   0.0|    2.0|46.0|30.0|             41.0|2016_10|
|100335346| 2774|    0|     0|      1|      0.0|       0.0|  693.2773|  0.0|   0.0|    0.5|49.0|30.0|             90.0|2016_10|
|160134740| 5845|    1|     1|      1|2183.1932| 2183.1932| 2183.1932|  2.0|   2.0|    2.0|13.0|30.0|   

Dinámica producto nuevo
==

Podemos obtener el dataframe de dinámica de producto nuevo

In [6]:
from athena2pyspark.athena_sql.dinamicas import producto_nuevo 
# “El código nunca miente, los comentarios sí” -- Ron Jeffries

#todo: agregar codigo_siebel
producto_nuevo_query = producto_nuevo(subclase=110209, marca='2717', lift=8) # creamos la query

In [8]:
s3_output = "s3://leonardo.exalitica.com/boto3/query_examples_dinamica_producto_nuevo/"

path_producto_nuevo = ath.run_query(query = producto_nuevo_query, 
                                    database = "prod_jumbo", 
                                    s3_output = s3_output, 
                                    spark = spark)

Execution ID: e556104a-ff37-4ba4-bd2f-2a40cbe9e637


In [12]:
path_producto_nuevo

's3://leonardo.exalitica.com/boto3/query_examples_dinamica_producto_nuevo/9f40d329-872c-4fb6-af62-435991668f66.csv'

In [10]:
df_producto_nuevo = ath.get_dataframe(path_query=path_producto_nuevo, spark = spark)
df_producto_nuevo.limit(10).show()

+---------+--------+---------------+-------------+------------+----------------+-------+--------------------+-----------+-----+
| party_id|promo_id|comm_channel_cd|codigo_siebel|codigo_motor|communication_id|page_id|   datos_de_contacto|correlativo|grupo|
+---------+--------+---------------+-------------+------------+----------------+-------+--------------------+-----------+-----+
|163543884|     300|              1|      PopCorn|         300|               1|      1|CLAUDIAGAZMURI@ED.CL|      61009|    0|
|112191543|     300|              1|      PopCorn|         300|               1|      1|leonel.gonzalez@t...|      61009|    0|
|140088569|     300|              1|      PopCorn|         300|               1|      1|   canuchj@gmail.com|      61009|    0|
|173584284|     300|              1|      PopCorn|         300|               1|      1|   HSALAZR@GMAIL.COM|      61009|    0|
|176209065|     300|              1|      PopCorn|         300|               1|      1|PACHICLARKE@HOTM

Generar el "create table" a partir del dataframe ya creado
==

Es util registrar este dataframe dentro de un catálogo para poder seguir haciendo consultas dentro de athena

In [12]:
s3_input = path_producto_nuevo
create_database, create_table = ath.get_ddl(df=df_producto_nuevo,
                                            database="ljofre",
                                            table="nueva_tabla_de_ejemplo",
                                            s3_input=s3_output)

In [15]:
print(create_database)

CREATE DATABASE IF NOT EXISTS ljofre;


In [ ]:
print(create_table)

Ejecutar create table y dejar registrada la tabla en Athena
==

“Cuando trabajo en un problema nunca pienso sobre la elegancia, sólo sobre cómo resolverlo. Pero cuando he acabado, si la solución no es elegante, sé que es incorrecta” 
-- R. Buckminster Fuller

In [ ]:
from athena2pyspark.config import aws_access_key_id, aws_secret_access_key

import boto3

client = boto3.client('athena', region_name='us-east-1', 
                      aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key)

response = client.start_query_execution(
    QueryString=create_table,
    QueryExecutionContext={
        'Database': "ljofre"
        },
    ResultConfiguration={
            'OutputLocation': s3_output,
            }
    )
print('Execution ID: ' + response['QueryExecutionId'])

Hacer un listado a partir de la prepriorizacion
==

In [ ]:
print(create_table)